R => Python  
Source: https://web.stanford.edu/class/stats191/notebooks/Penalized_regression.html


NOTE: 
* Not exactly porting R to Python.
* Also not [rpy2](https://rpy2.readthedocs.io/en/version_2.8.x/)

## Bias-variance tradeoff 

In [ ]:
import os
import numpy as np
import pandas as pd
from scipy import stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.formula.api import glm
from statsmodels.regression.linear_model import OLS
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score

In [ ]:
sm.__version__

In [ ]:
import scipy; scipy.__version__



- One goal of a regression analysis is to "build" a model that predicts well -- AIC / $C_p$ & Cross-validation selection criteria based on this.

- This is slightly different than the goal of making inferences about $\beta$ that we've focused on so far.

- What does "predict well" mean? 
$$
\begin{aligned}
     MSE_{pop}({{\cal M}}) &= {\mathbb{E}}\left((Y_{new} - \widehat{Y}_{new,{\cal M}}(X_{new}))^2\right) \\
     &=
     {\text{Var}}(Y_{new}) + {\text{Var}}(\widehat{Y}_{new,{\cal M}}) +
     \\
     & \qquad \quad \text{Bias}(\widehat{Y}_{new,{\cal M}})^2.
     \end{aligned}$$
 
- Can we take an estimator for a model ${\cal M}$ and make it better in terms of $MSE$?


## Shrinkage estimators: one sample problem

1. Generate $Y_{100 \times 1} \sim N(\mu \cdot 1, 5^2 I_{100 \times 100})$, with $\mu=0.5$.
2. For $0 \leq \alpha \leq 1$, set $\hat{Y}(\alpha) = \alpha \bar{Y}.$
3. Compute $MSE(\hat{Y}(\alpha)) = \frac{1}{100}\sum_{i=1}^{100} (\hat{Y}_{\alpha} - 0.5)^2$
4. Repeat 1000 times, plot average of $MSE(\hat{Y}(\alpha))$.

**For what value of $\alpha$ is $\hat{Y}(\alpha)$ unbiased?**

**Is this the best estimate of $\mu$ in terms of MSE?**

In [ ]:
mu = 0.5
sigma = 5
nsample = 100
ntrial = 10000

In [ ]:
def get_mse(mu_hat, mu):
    return np.sum((mu_hat - mu)**2) / len(mu)

In [ ]:
alpha = np.linspace(0, 1, 20)

In [ ]:
bias= (1 - alpha) * mu

In [ ]:
variance = (alpha**2) * 25/100

In [ ]:
mse = []
for trial in range(ntrial):
    result = []
    z = stats.norm.rvs(size=nsample) * sigma + mu
    for num in alpha:
        result.append(get_mse(num * np.mean(z) * np.ones(nsample), mu * np.ones(nsample)))
    mse.append(result)

In [ ]:
mse_mean = np.mean(mse, axis=0)

In [ ]:
mse_mean

In [ ]:
plt.plot(alpha, mse_mean, 'r')
plt.ylim(bottom=0)
plt.ylabel('MSE(alpha)')
plt.xlabel('Shrinkage factor, alpha')
plt.show()

In [ ]:
plt.plot(alpha, mse_mean, 'r')
plt.plot(alpha, bias**2, 'g')
plt.plot(alpha, variance, 'b')
plt.ylim(bottom=0)
plt.ylabel('MSE(alpha)')
plt.xlabel('Shrinkage factor, alpha')
plt.legend
plt.show()


## Shrinkage & Penalties

* Shrinkage can be thought of as "constrained" or "penalized" minimization.

* Constrained form:
$$\text{minimize}_{\mu} \sum_{i=1}^n (Y_i - \mu)^2 \quad \text{subject to $\mu^2 \leq C$}$$

* Lagrange multiplier form: equivalent to 
$$\widehat{\mu}_{\lambda} = \text{argmin}_{\mu} \sum_{i=1}^n (Y_i - \mu)^2 + \lambda \cdot \mu^2$$ for some $\lambda=\lambda_C$.

* As we vary $\lambda$ we solve all versions of the constrained form.

### Solving for $\widehat{\mu}_{\lambda}$

* Differentiating: $- 2 \sum_{i=1}^n (Y_i - \widehat{\mu}_{\lambda}) + 2 \lambda \widehat{\mu}_{\lambda} = 0$
* Solving $\widehat{\mu}_{\lambda} = \frac{\sum_{i=1}^n Y_i}{n + \lambda} = \frac{n}{n+\lambda} \overline{Y}.$
* As $\lambda \rightarrow 0$, $\widehat{\mu}_{\lambda} \rightarrow {\overline{Y}}.$
* As $\lambda \rightarrow \infty$ $\widehat{\mu}_{\lambda} \rightarrow 0.$

** We see that $\widehat{\mu}_{\lambda} = \bar{Y} \cdot \left(\frac{n}{n+\lambda}\right).$ **

** In other words, considering all penalized estimators traces out the
MSE curve above.**


In [ ]:
lam = nsample / alpha - nsample

In [ ]:
lam

In [ ]:
plt.plot(lam, mse_mean, 'r')
plt.ylim(bottom=0)
plt.ylabel('MSE(alpha)')
plt.xlabel('Penalty parameter, lambda')
plt.show()

In [ ]:
plt.plot(lam, mse_mean, 'r')
plt.plot(lam, bias**2, 'g')
plt.plot(lam, variance, 'b')
plt.ylim(bottom=0)
plt.ylabel('MSE(alpha)')
plt.xlabel('Penalty parameter, lambda')
plt.legend()
plt.show()

### How much to shrink?

- In our one-sample example,
- $$\begin{aligned}
 MSE_{pop}(\alpha) &=   {\text{Var}}( \alpha \bar{Y}) + \text{Bias}(\alpha \bar{Y})^2 +  \text{Var}(Y_{new})
\\
 &= \frac{\alpha^2 \sigma^2}{n} + \mu^2 (1 - \alpha)^2 +  \text{Var}(Y_{new}) 
 \end{aligned}$$
- Differentiating and solving: 
$$\begin{aligned}
 0 &= -2 \mu^2(1 - \alpha^*) + 2 \frac{\alpha^* \sigma^2}{n}  \\
 \alpha^* & = \frac{\mu^2}{\mu^2+\sigma^2/n} = \frac{(\mu/(\sigma/\sqrt{n}))^2}{(\mu/(\sigma/\sqrt{n}))^2 + 1} \\
 &= \frac{0.5^2}{0.5^2+25/100} = 0.5
 \end{aligned}$$
     
** We see that the optimal $\alpha$ depends on the unknown $SNR=\mu/(\sigma/\sqrt{n})$. Value is 1/8.**

** In practice we might hope to estimate MSE with cross-validation.**

Let's see how our theoretical choice matches the 
MSE on our 100 sample.

In [ ]:
plt.plot(alpha, mse_mean, 'r')
plt.ylim(bottom=0)
plt.ylabel('MSE(alpha)')
plt.xlabel('Shrinkage factor, alpha')
plt.axvline(mu**2/(mu**2+sigma**2/nsample), linestyle='--')
plt.show()

### Penalties & Priors

- Minimizing $\sum_{i=1}^n (Y_i - \mu)^2 + \lambda \mu^2$ is similar to computing "MLE" of $\mu$ if the likelihood was proportional to 
$$\exp \left(-\frac{1}{2\sigma^2}\left(  \|Y-\mu\|^2_2 + \lambda \mu^2\right) \right).$$

- If $\lambda=m$, an integer, then $\widehat{\mu}_{\lambda}$ is the sample mean of $(Y_1, \dots, Y_n,0 ,\dots, 0) \in \mathbb{R}^{n+m}$.

- This is equivalent to adding some data with $Y=0$. 

- To a Bayesian,
this extra data is a *prior distribution* and we are computing the so-called
*MAP* or posterior mode.

## AIC as penalized regression

- Model selection with $C_p$ (or AIC with $\sigma^2$ assumed known)
is a version of penalized regression.

- The best subsets version of AIC (which is not exactly equivalent to *step*)
$$
\hat{\beta}_{AIC} = \text{argmin}_{\beta} \frac{1}{\sigma^2}\|Y-X\beta\|^2_2 + 2 \|\beta\|_0
$$
where
$$
\|\beta\|_0 = \#\left\{j : \beta_j \neq 0 \right\}
$$
is called the $\ell_0$ norm.

- The $\ell_0$ penalty can be thought of as a measure of *complexity* of the model. Most penalties are similar versions of *complexity*.

## Penalized regression in general

* Not all biased models are better – we need a way to find "good" biased models.

* Inference ($F$, $\chi^2$ tests, etc) is not quite exact for biased models.
Though, there has been some recent work to address the issue of [post-selection inference](http://arxiv.org/abs/1311.6238), at least for some penalized regression problems.

* Heuristically, "large $\beta$" (measured by some norm) is interpreted as "complex model". Goal is really to penalize "complex" models, i.e. Occam’s razor.
* If truth really is complex, this may not work! (But, it will then be hard to build a good model anyways ... (statistical lore))

## Ridge regression

- Assume that columns $(X_j)_{1 \leq j \leq p}$ have zero mean, and SD 1 and $Y$ has zero mean.

- This is called the *standardized model*.

- The ridge estimator is
$$
\begin{aligned}
\hat{\beta}_{\lambda} &= \text{argmin}_{\beta} \frac{1}{2n}\|Y-X\beta\|^2_2 + \frac{\lambda}{2} \|\beta\|^2_2 \\
&= \text{argmin}_{\beta} MSE_{\lambda}(\beta)
\end{aligned}
$$
  
- Corresponds (through Lagrange multiplier) to a quadratic constraint on ${\beta_{}}$’s.

- This is the natural generalization of the penalized
version of our shrinkage estimator.

### Solving the normal equations

* Normal equations $$\frac{\partial}{\partial {\beta_{l}}} MSE_{\lambda}({\beta_{}}) = - \frac{1}{n}  (Y - X{\beta_{}})^TX_l  +  \lambda {\beta_{l}}$$
* $$- \frac{1}{n}(Y - X{\widehat{\beta}_{\lambda}})^T X_l +  \lambda {\widehat{\beta}_{l,\lambda}} = 0, \qquad 1 \leq l \leq p$$
* In matrix form $$-\frac{X^TY}{n} +  \left(\frac{X^TX}{n} + \lambda I\right) {\widehat{\beta}_{\lambda}} = 0.$$
* Or $${\widehat{\beta}_{\lambda}} = \left(\frac{X^TX}{n} + \lambda I\right)^{-1} \frac{X^TY}{n}.$$


### Ridge regression

In [ ]:
from sklearn import datasets
diabetes = datasets.load_diabetes()
"""
From Bradley Efron, Trevor Hastie, Iain Johnstone and Robert Tibshirani (2004) 
"Least Angle Regression," Annals of Statistics (with discussion), 407-499, we have

"Ten baseline variables, age, sex, body mass index, average blood pressure, 
and six blood serum measurements were obtained for each of n = 442 diabetes patients, 
as well as the response of interest, a quantitative measure of disease progression one year 
after baseline."

In the tab delimited file above, the variables are named

    AGE SEX BMI BP S1 S2 S3 S4 S5 S6 Y

reference: https://www4.stat.ncsu.edu/~boos/var.select/diabetes.html
"""

In [ ]:
# Sklearn version -> Standardized: zero mean and unit L2 norm 
diabetes.data[:5]

In [ ]:
diabetes.data

In [ ]:
# Sklearn version -> Standardized: zero mean and unit L2 norm 
diabetes.target[:5]

In [ ]:
data_with_const = sm.add_constant(diabetes.data)

** The following process and the plot is different from R notebook**

In [ ]:
# The fraction of the penalty given to the L1 penalty term. 
# Must be between 0 and 1 (inclusive). 
# If 0, the fit is a ridge fit, if 1 it is a lasso fit.
model = OLS(diabetes.target, data_with_const).fit_regularized(alpha=0.0, 
                                                            L1_wt=0.0)

In [ ]:
model.params

![R_result](./img/13_lm_ridge_coef.png "R result")

** The following process and the plot is different from R notebook**

In [ ]:
alpha_lst = np.linspace(0, 0.01, 100)

In [ ]:
result = []
for alpha in alpha_lst:
    model = OLS(diabetes.target, data_with_const).fit_regularized(alpha=alpha, 
                                                            L1_wt=0.0)
    result.append(model.params)

In [ ]:
df = pd.DataFrame(result, index=alpha_lst)
ax = df.plot()
ax.set_xlabel('alpha')
ax.set_ylabel('params')

** The above process and the plot is different from R notebook**  
It seems like to me that the plot in the R notebook used ```diabetes.ridge['coef']```, not ```coef(diabetes.ridge)```

As ```lm.ridge``` says (in describing the ```$coef``` element of the returned object) [emphasis added]

>coef: matrix of coefficients, one row for each value of ‘lambda’. Note that these are not on the original scale and are for use by the ‘coef’ method.

This means, specifically, that the ```$coef``` element is not intended for end-users ("if you have to ask ..."). (If you want to see how ```$coef``` is translated, inspect ```MASS:::coef.ridgelm```.) In general, it's better practice to use an accessor method such as ```coef()```, when it exists, than to extract components from the guts of a returned object using ```$``` (or ```@``` for S4 objects) - for exactly this reason. Package authors provide ```coef()``` methods for a reason ...

[Source](https://stackoverflow.com/questions/34567594/masslm-ridge-coefficients)

### Choosing $\lambda$

* If we knew $E[MSE_{\lambda}]$ as a function of $\lambda$ then we would simply choose the $\lambda$ that minimizes it.
* To do this, we need to estimate it.
* A popular method is cross-validation as a function of $\lambda$. Breaks the data up into smaller groups and uses part of the data to predict the rest.
* We saw this in diagnostics (Cook’s distance measured the fit with and without each point in the data set) and model selection.

### $K$-fold cross-validation for penalized model

* Fix a model (i.e. fix $\lambda$). Break data set into $K$ approximately equal sized groups $(G_1, \dots, G_K)$.
* for (i in 1:K)
   Use all groups except $G_i$ to fit model, predict outcome in group $G_i$ based on this model $\widehat{Y}_{j(i),\lambda}, j \in G_i$.
* Estimate $CV(\lambda) = \frac{1}{n}\sum_{i=1}^K \sum_{j \in G_i} (Y_j - \widehat{Y}_{j(i),\lambda})^2.$

Here is a function to estimate the CV for our one parameter example. In practice, we only have one sample to form the CV curve. In this example below,
I will compute the average CV error for 500 trials to show that it is roughly
comparable in shape to the MSE curve.

In [ ]:
# Python port for glmnet
# https://web.stanford.edu/~hastie/glmnet_python/
# https://github.com/bbalasub1/glmnet_python
# Algorithm was designed by Jerome Friedman, Trevor Hastie and Rob Tibshirani. Fortran code was written by Jerome Friedman. R wrapper (from which the MATLAB wrapper was adapted) was written by Trevor Hastie.
# The original MATLAB wrapper was written by Hui Jiang (14 Jul 2009), and was updated and is maintained by Junyang Qian (30 Aug 2013).
# This python wrapper (which was adapted from the MATLAB and R wrappers) was originally written by B. J. Balakumar (5 Sep 2016).
# List of other contributors along with a summary of their contributions is included in the contributors.dat file.
# B. J. Balakumar, bbalasub@gmail.com (Sep 5, 2016). Department of Statistics, Stanford University, Stanford, CA

# There are other glmnet pkgs..
# https://github.com/civisanalytics/python-glmnet